In [60]:
# import csv

# csv_file_name = 'MIS_data.csv'

# columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]
# with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=columns)
#     writer.writeheader()

# def add_row_to_csv(data):
#     with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
#         writer = csv.DictWriter(file, fieldnames=columns)
#         writer.writerow(data)

In [61]:
import csv

csv_file_name = 'MIS_data.csv'

columns = ["Title", "Authors", "Publish Date", "Abstract", "Keywords", "PDF Name"]

def add_row_to_csv(data):
    with open(csv_file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writerow(data)

In [62]:
def get_credentials():
    with open('../credentials.txt', 'r') as file:
        lines = file.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
        file.close()
        return username, password


In [63]:
def overwrite_line(line_number, new_value):
    with open('progress.txt', 'r') as file:
        lines = file.readlines()
        
    if 0 <= line_number < len(lines):
        lines[line_number] = f"{new_value}\n" 
        with open('progress.txt', 'w') as file:
            file.writelines(lines)
        return True
    else:
        return False

def read_numbers():
    with open('progress.txt', 'r') as file:
        numbers = [int(line.strip()) for line in file]
    return numbers[0], numbers[1], numbers[2], numbers[3]

In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
import urllib
import os
import shutil
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Initial_path = r"C:\Users\yitong\Desktop\Research-Assistant\Scrape\MIS\PDF"

In [65]:
def get_one_article(driver, wait):

    # Function to find the following dd for a given dt text
    def find_following_dd_by_dt_text(dt_text):
        dt_elements = driver.find_elements(By.XPATH, "//dl[@id='citationFields']/dt")
        for dt in dt_elements:
            if dt.text.strip(':') == dt_text:
                return dt.find_element(By.XPATH, "./following-sibling::dd[1]")
        return None

    # Extract Title
    title_element = driver.find_element(By.CSS_SELECTOR, "dd.citation-title > a > div > h1")
    title = title_element.text

    # Extract Authors
    authors_element = find_following_dd_by_dt_text("Authors")
    authors = "; ".join([author.text for author in authors_element.find_elements(By.TAG_NAME, "a")])

    # Extract Subject Terms
    subject_terms_element = find_following_dd_by_dt_text("Subject Terms")
    subject_terms = "; ".join([term.text for term in subject_terms_element.find_elements(By.TAG_NAME, "a")])

    # Extract Publish Date (assuming it's within the Source information)
    source_element = find_following_dd_by_dt_text("Source")
    publish_date = source_element.text.split('. ')[1].split(', ')[0]

    # Extract Accession Number
    accession_number_element = find_following_dd_by_dt_text("Accession Number")
    accession_number = accession_number_element.text

    abstract_element = find_following_dd_by_dt_text("Abstract")
    abstract = abstract_element.text

    add_row_to_csv({
        "Title": title,
        "Authors": authors,
        "Publish Date": publish_date,
        "Abstract": abstract,
        "Keywords": subject_terms,
        "PDF Name": accession_number
    })

    pdf_link_element = driver.find_element("link text", 'PDF Full Text')
    pdf_link_element.click()
    wait.until(EC.element_to_be_clickable((By.ID, "downloadLink")))
    pdf_download_element = driver.find_element(By.ID, "downloadLink")
    pdf_download_element.click()
    time.sleep(10)
    filename = max([Initial_path + "\\" + f for f in os.listdir(Initial_path)], key=os.path.getctime)
    shutil.move(filename, os.path.join(Initial_path, accession_number + ".pdf"))
    window_handles = driver.window_handles
    driver.switch_to.window(window_handles[0])
    time.sleep(5)
    driver.execute_script("window.history.go(-1)")
    time.sleep(10)
    driver.execute_script("window.history.go(-1)")
    time.sleep(10)
    print("back to page page")




In [66]:
# 先处理这个年份这个issue剩下的article
# 再处理这个年份剩下的issue
# 再处理剩下的年份

def recover_from_breakpoint(year, issue, page, article):
    
    user_id, password = get_credentials()

    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": Initial_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True 
    })

    wd = webdriver.Chrome(service = Service(r'../chromedriver.exe'), options=options)
    wait = WebDriverWait(wd, 60)
    
    wd.get("https://libproxy1.nus.edu.sg/login?url=https://search.ebscohost.com/login.aspx?direct=true&db=bsu&jid=MIS&site=ehost-live")
    button = wd.find_element(By.ID, 'btn_nus')
    button.click()
    wait.until(EC.element_to_be_clickable((By.ID, 'userNameInput')))
    user_id_box = wd.find_element(By.ID, 'userNameInput')
    password_box = wd.find_element(By.ID, 'passwordInput')
    user_id_box.send_keys(user_id)
    password_box.send_keys(password)
    submit_button = wd.find_element(By.ID, 'submitButton')
    submit_button.click()
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "osano-cm-accept-all")))
    cookie_button_element = wd.find_element(By.CLASS_NAME, "osano-cm-accept-all")
    cookie_button_element.click()

    year_xpath = "//td[@class='authVolIssue_volume_cell']/a[contains(@href, 'javascript:ToggleVolumeIssue')]"
    wait.until(EC.element_to_be_clickable((By.XPATH, year_xpath)))
    year_elements = wd.find_elements(By.XPATH, year_xpath)
    for j in range(year, len(year_elements)):
        year_elements[j].click()
        time.sleep(3)
        issue_xpath = "//a[@data-auto='volume_issue_issue_link' and contains(@href, 'XslPostBack')]"
        wait.until(EC.element_to_be_clickable((By.XPATH, issue_xpath)))
        issue_elements = wd.find_elements(By.XPATH, issue_xpath)
        for i in range(issue, len(issue_elements)):
            issue_elements[i].click()
            time.sleep(5)
            if page == 1:
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for k in range(article, len(title_elements)):
                    title_elements[k].click()
                    print(year, issue, page, k)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    overwrite_line(3, k + 1)
                page = 2
                article = 0
                overwrite_line(2, page)
                overwrite_line(3, 0)
                # raise Exception

            if page == 2:
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 2']")
                page_element[0].click()
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for m in range(article, len(title_elements)):
                    title_elements[m].click()
                    print(year, issue, page, m)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    overwrite_line(3, m + 1)

                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 3']")
                if len(page_element) == 0:
                    # This issue only has 2 pages
                    page = 1
                    article = 0
                    overwrite_line(2, page)
                    overwrite_line(3, 0)
                    page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 1']")
                    page_element[0].click()
                    print("back to page 1")
                    break
                
                page = 3
                article = 0
                overwrite_line(2, page)
                overwrite_line(3, 0)

            if page == 3:
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 3']")
                page_element[0].click()
                title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                for n in range(article, len(title_elements)):
                    title_elements[n].click()
                    print(year, issue, page, n)
                    time.sleep(5)
                    get_one_article(wd, wait)
                    window_handles = wd.window_handles
                    wd.switch_to.window(window_handles[0])
                    title_elements = wd.find_elements(By.CSS_SELECTOR, "a.title-link.color-p4")
                    overwrite_line(3, n + 1)
                page_element = wd.find_elements(By.XPATH, "//a[@title='Skip to page 1']")
                page_element[0].click()
                print("back to page 1")

            page = 1
            article = 0
            overwrite_line(2, page)
            overwrite_line(3, 0)
            overwrite_line(1, i + 1)
            wd.get("https://libproxy1.nus.edu.sg/login?url=https://search.ebscohost.com/login.aspx?direct=true&db=bsu&jid=MIS&site=ehost-live")
            time.sleep(10)
            print("back to issue page")
            year_elements = wd.find_elements(By.XPATH, year_xpath)
            year_elements[j].click()
            time.sleep(5)
            issue_elements = wd.find_elements(By.XPATH, issue_xpath)
        issue = 0
        overwrite_line(1, 0)
        overwrite_line(0, j + 1)


In [67]:
while True:
    try:
        year, issue, page, article = read_numbers()
        recover_from_breakpoint(year, issue, page, article)
    except Exception as e:
        print(e)
        print("Failed")

Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF6C1905E42+3538674]
	(No symbol) [0x00007FF6C1524C02]
	(No symbol) [0x00007FF6C13D5AEB]
	(No symbol) [0x00007FF6C13B288C]
	(No symbol) [0x00007FF6C1445DD7]
	(No symbol) [0x00007FF6C145B40F]
	(No symbol) [0x00007FF6C143EE53]
	(No symbol) [0x00007FF6C140F514]
	(No symbol) [0x00007FF6C1410631]
	GetHandleVerifier [0x00007FF6C1936CAD+3738973]
	GetHandleVerifier [0x00007FF6C198C506+4089270]
	GetHandleVerifier [0x00007FF6C1984823+4057299]
	GetHandleVerifier [0x00007FF6C1655C49+720121]
	(No symbol) [0x00007FF6C153126F]
	(No symbol) [0x00007FF6C152C304]
	(No symbol) [0x00007FF6C152C432]
	(No symbol) [0x00007FF6C151BD04]
	BaseThreadInitThunk [0x00007FFFDAE0257D+29]
	RtlUserThreadStart [0x00007FFFDCE6AA58+40]

Failed
